# HarmonyHub

In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
from datetime import datetime

# Create a sample music database
def create_music_database():
    genres = ['Pop', 'Rock', 'Hip-Hop', 'Electronic', 'R&B', 'Jazz', 'Classical', 'Country', 'Metal', 'Indie']
    artists = {
        'Pop': ['Taylor Swift', 'Ariana Grande', 'Ed Sheeran', 'Dua Lipa', 'Justin Bieber'],
        'Rock': ['The Beatles', 'Led Zeppelin', 'Queen', 'Pink Floyd', 'The Rolling Stones'],
        'Hip-Hop': ['Kendrick Lamar', 'Drake', 'J. Cole', 'Travis Scott', 'Eminem'],
        'Electronic': ['Daft Punk', 'The Chemical Brothers', 'Deadmau5', 'Calvin Harris', 'Martin Garrix'],
        'R&B': ['Beyoncé', 'The Weeknd', 'Frank Ocean', 'SZA', 'Usher'],
        'Jazz': ['Miles Davis', 'John Coltrane', 'Louis Armstrong', 'Ella Fitzgerald', 'Duke Ellington'],
        'Classical': ['Mozart', 'Beethoven', 'Bach', 'Chopin', 'Tchaikovsky'],
        'Country': ['Johnny Cash', 'Dolly Parton', 'Willie Nelson', 'Taylor Swift', 'Luke Combs'],
        'Metal': ['Metallica', 'Iron Maiden', 'Black Sabbath', 'Slipknot', 'System of a Down'],
        'Indie': ['Arctic Monkeys', 'The Strokes', 'Tame Impala', 'Vampire Weekend', 'Florence + The Machine']
    }
    
    songs = []
    for genre in genres:
        for artist in artists[genre]:
            # Create 2 songs per artist
            for i in range(1, 3):
                year = random.randint(1960, 2023)
                duration = random.randint(120, 360)  # 2-6 minutes in seconds
                popularity = random.randint(1, 100)
                
                songs.append({
                    'Title': f"{artist.split()[0]}'s Song {i}",
                    'Artist': artist,
                    'Genre': genre,
                    'Year': year,
                    'Duration': duration,
                    'Popularity': popularity,
                    'ID': len(songs) + 1
                })
    
    return pd.DataFrame(songs)

# Initialize the music database
music_db = create_music_database()

# Function to format duration (seconds to mm:ss)
def format_duration(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    return f"{minutes}:{seconds:02d}"

# Search functions
def search_by_title(title):
    return music_db[music_db['Title'].str.contains(title, case=False)]

def search_by_artist(artist):
    return music_db[music_db['Artist'].str.contains(artist, case=False)]

def search_by_genre(genre):
    return music_db[music_db['Genre'].str.contains(genre, case=False)]

def search_by_year_range(start_year, end_year):
    return music_db[(music_db['Year'] >= start_year) & (music_db['Year'] <= end_year)]

def search_by_duration(max_duration):
    return music_db[music_db['Duration'] <= max_duration]

# Recommendation function
def recommend_songs(genre=None, artist=None, min_popularity=50):
    if genre and artist:
        filtered = music_db[(music_db['Genre'] == genre) & (music_db['Artist'] == artist)]
    elif genre:
        filtered = music_db[music_db['Genre'] == genre]
    elif artist:
        filtered = music_db[music_db['Artist'] == artist]
    else:
        filtered = music_db.copy()
    
    filtered = filtered[filtered['Popularity'] >= min_popularity]
    if len(filtered) > 5:
        return filtered.sample(5)
    return filtered

# Widgets for the UI
title_search = widgets.Text(placeholder='Enter song title', description='Title:')
artist_search = widgets.Text(placeholder='Enter artist name', description='Artist:')
genre_dropdown = widgets.Dropdown(options=['All'] + sorted(music_db['Genre'].unique()), description='Genre:')
year_slider = widgets.IntRangeSlider(value=[1960, 2023], min=1960, max=2023, description='Year Range:')
duration_slider = widgets.IntSlider(value=300, min=120, max=600, description='Max Duration (sec):')
search_button = widgets.Button(description="Search Songs")
recommend_button = widgets.Button(description="Get Recommendations")
output = widgets.Output()

# Search button click handler
def on_search_button_clicked(b):
    with output:
        clear_output()
        results = music_db.copy()
        
        # Apply filters
        if title_search.value:
            results = results[results['Title'].str.contains(title_search.value, case=False)]
        if artist_search.value:
            results = results[results['Artist'].str.contains(artist_search.value, case=False)]
        if genre_dropdown.value != 'All':
            results = results[results['Genre'] == genre_dropdown.value]
        if year_slider.value != [1960, 2023]:
            results = results[(results['Year'] >= year_slider.value[0]) & 
                              (results['Year'] <= year_slider.value[1])]
        if duration_slider.value != 600:
            results = results[results['Duration'] <= duration_slider.value]
        
        if len(results) == 0:
            print("No songs found matching your criteria.")
        else:
            print(f"Found {len(results)} songs:")
            display(results[['Title', 'Artist', 'Genre', 'Year', 'Duration']].head(20))
            print("\nTip: Use more specific filters if you see too many results.")

# Recommendation button click handler
def on_recommend_button_clicked(b):
    with output:
        clear_output()
        genre = genre_dropdown.value if genre_dropdown.value != 'All' else None
        artist = artist_search.value if artist_search.value else None
        
        recommendations = recommend_songs(genre=genre, artist=artist)
        
        if len(recommendations) == 0:
            print("No recommendations found. Try broadening your search.")
        else:
            print("Recommended songs for you:")
            display(recommendations[['Title', 'Artist', 'Genre', 'Year', 'Popularity']])

# Set button click handlers
search_button.on_click(on_search_button_clicked)
recommend_button.on_click(on_recommend_button_clicked)

# Display the UI
print("🎵 Music Discovery Tool 🎵")
print("Search for songs or get recommendations based on your preferences.")
print(f"Database contains {len(music_db)} songs from {len(music_db['Artist'].unique())} artists.\n")

display(widgets.HBox([title_search, artist_search]))
display(genre_dropdown)
display(year_slider)
display(duration_slider)
display(widgets.HBox([search_button, recommend_button]))
display(output)

# Display some random songs as examples
print("\nHere are some songs from our database:")
sample_songs = music_db.sample(5)
for _, song in sample_songs.iterrows():
    print(f"{song['Title']} by {song['Artist']} ({song['Year']}) - {format_duration(song['Duration'])}")

🎵 Music Discovery Tool 🎵
Search for songs or get recommendations based on your preferences.
Database contains 100 songs from 49 artists.



Dropdown(description='Genre:', options=('All', 'Classical', 'Country', 'Electronic', 'Hip-Hop', 'Indie', 'Jazz…

IntRangeSlider(value=(1960, 2023), description='Year Range:', max=2023, min=1960)

IntSlider(value=300, description='Max Duration (sec):', max=600, min=120)

Output()


Here are some songs from our database:
Deadmau5's Song 1 by Deadmau5 (1976) - 2:30
Beyoncé's Song 1 by Beyoncé (2015) - 4:30
Beethoven's Song 1 by Beethoven (1970) - 4:44
Arctic's Song 1 by Arctic Monkeys (1969) - 2:07
Kendrick's Song 1 by Kendrick Lamar (1991) - 4:50
